In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification

from imblearn.over_sampling import RandomOverSampler

import pandas as pd
import numpy as np

In [2]:
file = pd.read_csv("database.csv") 
df= file[["Type", 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', "Tool wear [min]", "TWF", "HDF", "PWF", "OSF", "RNF"]] 

X = df[["Type", 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', "Tool wear [min]"]] 
Y = df[["TWF", "HDF", "PWF", "OSF", "RNF"]] 
Y_strat = Y.sum(axis=1) 

df_twf = df[(df["TWF"] == 1) & 
          (df["HDF"] == 0) & 
          (df["PWF"] == 0) & 
          (df["OSF"] == 0) & 
          (df["RNF"] == 0)]

df_hdf = df[(df["TWF"] == 0) & 
          (df["HDF"] == 1) & 
          (df["PWF"] == 0) & 
          (df["OSF"] == 0) & 
          (df["RNF"] == 0)]

df_pwf = df[(df["TWF"] == 0) & 
          (df["HDF"] == 0) & 
          (df["PWF"] == 1) & 
          (df["OSF"] == 0) & 
          (df["RNF"] == 0)]

df_osf = df[(df["TWF"] == 0) & 
          (df["HDF"] == 0) & 
          (df["PWF"] == 0) & 
          (df["OSF"] == 1) & 
          (df["RNF"] == 0)]

df_rnf = df[(df["TWF"] == 0) & 
          (df["HDF"] == 0) & 
          (df["PWF"] == 0) & 
          (df["OSF"] == 0) & 
          (df["RNF"] == 1)]
print(len(df_twf), len(df_hdf), len(df_pwf), len(df_osf), len(df_rnf))

42 106 80 78 18


In [ ]:
df_2to = df[(df["TWF"] == 1) & 
          (df["HDF"] == 0) & 
          (df["PWF"] == 0) & 
          (df["OSF"] == 1) & 
          (df["RNF"] == 0)]

df_2tr = df[(df["TWF"] == 1) & 
          (df["HDF"] == 0) & 
          (df["PWF"] == 0) & 
          (df["OSF"] == 0) & 
          (df["RNF"] == 1)]

df_2hp = df[(df["TWF"] == 0) & 
          (df["HDF"] == 1) & 
          (df["PWF"] == 1) & 
          (df["OSF"] == 0) & 
          (df["RNF"] == 0)]

df_2ho = df[(df["TWF"] == 0) & 
          (df["HDF"] == 1) & 
          (df["PWF"] == 0) & 
          (df["OSF"] == 1) & 
          (df["RNF"] == 0)]

df_2po = df[(df["TWF"] == 0) & 
          (df["HDF"] == 0) & 
          (df["PWF"] == 1) & 
          (df["OSF"] == 1) & 
          (df["RNF"] == 0)]


print(len(df_2to), len(df_2tr), len(df_2hp), len(df_2ho), len(df_2po))

2 1 3 6 11


In [ ]:
df_3 = df[(df["TWF"] == 1) & 
          (df["HDF"] == 0) & 
          (df["PWF"] == 1) & 
          (df["OSF"] == 1) & 
          (df["RNF"] == 0)]
print(len(df_3))

1


In [11]:
n_muestras_a_agregar = 106-18
X_minoritaria=df[df["RNF"]==1]
y_minoritaria=Y[Y['RNF']==1]
rng = np.random.default_rng(seed=42)
# Realizar sobremuestreo aleatorio
indices_aleatorios = rng.integers(0, len(X_minoritaria), size=n_muestras_a_agregar)
X_minoritaria_sobremuestreada = X_minoritaria.iloc[indices_aleatorios]
y_minoritaria_sobremuestreada =y_minoritaria.iloc[indices_aleatorios]
X_sobremuestreado = pd.concat([X, X_minoritaria_sobremuestreada], ignore_index=True)
y_sobremuestreado = pd.concat([Y, y_minoritaria_sobremuestreada], ignore_index=True)

